In [2]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
tasas=pd.read_excel('../data/FEDFUNDS.xls',header=10)
cotizaciones_2008=pd.read_excel('../output/ITCRM completo.xlsx',sheet_name='cotizaciones')

In [4]:
tasas['mes']=tasas.observation_date.apply(lambda x: x.month)
tasas['anio']=tasas.observation_date.apply(lambda x: x.year)

In [5]:
cotizaciones_usd=pd.read_excel('../output/cotizaciones completas.xlsx', sheet_name=1).drop('Estados Unidos',axis=1)
cotizaciones_usd['Argentina']=cotizaciones_usd.merge(cotizaciones_2008[['Período','Estados Unidos']], on='Período',how='left')['Estados Unidos']
cotizaciones_usd['Período'] = pd.to_datetime(cotizaciones_usd['Período'], format='%d/%m/%Y')
cotizaciones_usd['mes']=cotizaciones_usd.Período.apply(lambda x: x.month)
cotizaciones_usd['anio']=cotizaciones_usd.Período.apply(lambda x: x.year)
cotizaciones_usd=cotizaciones_usd.groupby(['anio','mes'],as_index=False).mean()
cotizaciones_usd=cotizaciones_usd.merge(tasas[['anio','mes','FEDFUNDS']], on=['anio',"mes"],how='left')
cotizaciones_usd['Período']=pd.to_datetime('1/'+cotizaciones_usd.mes.astype(str)+'/'+cotizaciones_usd.anio.astype(str), format='%d/%m/%Y')

lista_paises=cotizaciones_usd.columns[2:]

var_cotizaciones_usd=cotizaciones_usd.copy()

for i in range(len(lista_paises)-1):
  var_cotizaciones_usd[lista_paises[i]]=var_cotizaciones_usd[lista_paises[i]].pct_change(12)

var_cotizaciones_usd.dropna(inplace=True)

var_cotizaciones_usd['FEDFUNDS']=cotizaciones_usd['FEDFUNDS']/10

var_cotizaciones_usd

,anio,mes,Brasil,Canada,Chile,China,Euro,Gran Bretaña,India,Japon,Mexico,Suiza,Uruguay,Vietnam,Argentina,FEDFUNDS,Período
12,2009,1,0.302199,0.212534,0.300948,-0.055820,0.112041,0.361420,-0.163000,-0.161253,0.272357,0.023095,0.100689,0.084271,0.101289,0.015,2009-01-01
13,2009,2,0.340389,0.246837,0.296663,-0.046085,0.152490,0.362423,-0.132439,-0.132515,0.355866,0.068510,0.108293,0.053546,0.112142,0.022,2009-02-01
14,2009,3,0.358317,0.267095,0.346058,-0.033875,0.193133,0.417699,-0.031228,-0.030665,0.365604,0.142696,0.161874,0.095678,0.158368,0.018,2009-03-01
15,2009,4,0.309648,0.207843,0.303477,-0.023915,0.194551,0.345223,-0.039645,-0.038624,0.276889,0.130788,0.210819,0.089783,0.165836,0.015,2009-04-01
16,2009,5,0.243333,0.152367,0.198622,-0.021187,0.138819,0.272313,-0.075269,-0.074826,0.262116,0.059359,0.195393,0.053056,0.182746,0.018,2009-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2022,5,-0.062938,0.059737,0.195033,0.041125,0.148117,0.131620,0.180720,0.055239,0.002964,0.085383,-0.071647,0.004748,0.251533,0.077,2022-05-01
173,2022,6,0.001833,0.047209,0.189451,0.042282,0.139692,0.136989,0.216753,0.061901,-0.001237,0.068830,-0.087555,0.011626,0.288577,0.121,2022-06-01
174,2022,7,0.040561,0.032814,0.265120,0.039905,0.161579,0.149033,0.239508,0.068012,0.028108,0.056558,-0.061668,0.016375,0.334777,0.168,2022-07-01
175,2022,8,-0.019500,0.025385,0.157600,0.050152,0.162674,0.152862,0.232386,0.073098,0.002508,0.047403,-0.063575,0.024183,0.391861,0.233,2022-08-01


In [14]:
desde=var_cotizaciones_usd.Período.iloc[0].strftime('%m/%Y')
hasta=var_cotizaciones_usd.Período.iloc[-1].strftime('%m/%Y')
desde

'01/2009'

In [15]:
def plot_tasa_deva():
    x = var_cotizaciones_usd["Período"]

    #Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(name = "USD-Real", x=x, y=var_cotizaciones_usd["Brasil"], mode = "lines"))
    cotizaciones_plot.add_trace(go.Scatter(name = "USD-Yuan", x=x, y=var_cotizaciones_usd["China"], mode = "lines"))
    # cotizaciones_plot.add_trace(go.Scatter(name = "ARS-USD", x=x, y=var_cotizaciones_usd["Argentina"], mode = "lines"))
    cotizaciones_plot.add_trace(go.Scatter(name = "USD-Euro", x=x, y=var_cotizaciones_usd["Euro"], mode = "lines"))
    cotizaciones_plot.add_trace(go.Scatter(name = "Tasa FED (x10)", x=x, y=var_cotizaciones_usd["FEDFUNDS"], mode = "lines"))
    #Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    cotizaciones_plot.update_yaxes(title_text="Tasa de devaluación", tickformat= ',.0%')
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
    height=600, width=1000,
    template = 'none',
    title_text=f"Tasa de devaluación de las monedas de los tres principales socios comerciales <br><sup>Variación interanual con frecuencia mensual. {desde}-{hasta}",
    legend = dict(yanchor="top",y=1.05, xanchor="left",x=0.25,orientation='h'))

    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.1,yref='paper', y=-0.1, 
                            xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    #Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
        xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot

plot_tasa_deva()

In [38]:
plot_tasa_deva().write_html('tasa_deva.html')